# The Semi-Supervised VAE

The semi-supervised setting represents an interesting intermediate case where some of the data is labeled and some is not. It is also of great practical importance, since we often have very little labeled data and much more unlabeled data. We’d clearly like to leverage labeled data to improve our models of the unlabeled data.

The semi-supervised setting is also well suited to generative models, where missing data can be accounted for quite naturally—at least conceptually. As we will see, in restricting our attention to semi-supervised generative models, there will be no shortage of different model variants and possible inference strategies. Although we’ll only be able to explore a few of these variants in detail, hopefully you will come away from the tutorial with a greater appreciation for the abstractions and modularity offered by probabilistic programming.

So let’s go about building a generative model. We have a dataset $D$ with $N$ datapoints,

$ D = $ {(xi, yi)}$  $

where the  {xi} are always observed and the labels {yi} are only observed for some subset of the data. Since we want to be able to model complex variations in the data, we’re going to make this a latent variable model with a local latent variable {zi} private to each pair (xi, yi). Even with this set of choices, a number of model variants are possible: we’re going to focus on the model variant depicted in Figure 1.

<img src="ss_vae_m2.png"  width="180" height="200">

For convenience—and since we’re going to model MNIST in our experiments below—let’s suppose the {xi} are images and the {yi} are digit labels. In this model setup, the latent random variable {zi} and the (partially observed) digit label jointly generate the observed image. The {zi} represents everything but the digit label, possibly handwriting style or position. Let’s sidestep asking when we expect this particular factorization of (xi, yi, zi) to be appropriate, since the answer to that question will depend in large part on the dataset in question (among other things). Let’s instead highlight some of the ways that inference in this model will be challenging as well as some of the solutions that we’ll be exploring in the rest of the tutorial.

# The Challenge of Inference

For concreteness we’re going to continue to assume that the partially-observed {yi} are discrete labels; we will also assume that the {zi} are continuous.

- If we apply the general recipe for stochastic variational inference to our model (see SVI Part I) we would be sampling the discrete (and thus non-reparameterizable) variable {yi} whenever it’s unobserved. As discussed in SVI Part III this will generally lead to high-variance gradient estimates.

- A common way to ameliorate this problem—and one that we’ll explore below—is to forego sampling and instead sum out all ten values of the class label {yi} when we calculate the ELBO for an unlabeled datapoint {xi} . This is more expensive per step, but can help us reduce the variance of our gradient estimator and thereby take fewer steps.

- Recall that the role of the guide is to ‘fill in’ latent random variables. Concretely, one component of our guide will be a digit classifier $q\phi(y|x)$ that will randomly ‘fill in’ labels {yi} given an image {xi}. Crucially, this means that the only term in the ELBO that will depend on $q\phi(.|x)$ is the term that involves a sum over unlabeled datapoints. This means that our classifier $q\phi(.|x)$—which in many cases will be the primary object of interest—will not be learning from the labeled datapoints (at least not directly).

- This seems like a potential problem. Luckily, various fixes are possible. Below we’ll follow the approach in reference, which involves introducing an additional objective function for the classifier to ensure that the classifier learns directly from the labeled data.
